# 用yfinance抓取股票資料，並匯出Excel檔案。

In [31]:
import yfinance as yf
import pandas as pd

# 設定股票代號與名稱對應
tickers = {'0050.TW': '元大台灣50', '006208.TW': '富邦台50'}

# 下載股票資料
data_frames = []
for ticker, name in tickers.items():
    data = yf.download(ticker, start='2019-01-01', end='2024-12-30')

    # 重新命名欄位
    data.columns = ['Open', 'High', 'Low', 'Close', 'AdjClose', 'Volume']

    # 計算當日均價 (以 (高+低+收盤)/3 作為均價的簡單估算)
    data['AvgPrice'] = (data['High'] + data['Low'] + data['Close']) / 3

    # 將成交股數 / 1000
    data['Volume'] = data['Volume'] / 1000

    # 將索引轉換為 yyyymmdd 格式並重設索引
    data.index = data.index.strftime('%Y-%m-%d')
    data.index.name = 'Date'
    data = data.reset_index()

    # 移除 .TW 並加入證券代碼與名稱
    ticker_clean = ticker.replace('.TW', '')  # 移除 .TW
    data['證券代碼'] = f"{ticker_clean} ({name})"

    # 選擇所需欄位
    data = data[['證券代碼', 'Date', 'AvgPrice', 'Volume']]

    # 重新命名欄位
    data.columns = ['證券代碼', '年月日', '當日均價(元)', '當日成交張數']

    # 加入到總資料中
    data_frames.append(data)

# 合併兩檔資料
final_data = pd.concat(data_frames, ignore_index=True)

# 轉換「年月日」為 datetime 格式，進行排序
final_data['年月日'] = pd.to_datetime(final_data['年月日'])

# 依「年月日」與「證券代碼」進行排序
final_data = final_data.sort_values(by=['年月日', '證券代碼']).reset_index(drop=True)

# 再將「年月日」轉回字串格式（yyyy-mm-dd）
final_data['年月日'] = final_data['年月日'].dt.strftime('%Y-%m-%d')

# 匯出為 CSV 檔案
final_data.to_csv('台股0050_006208_歷史資料.csv', index=False, encoding='utf-8-sig')

# 顯示前五筆資料
# final_data.head()
final_data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


,證券代碼,年月日,當日均價(元),當日成交張數
0,0050 (元大台灣50),2019-01-02,74.633334,8532.073
1,006208 (富邦台50),2019-01-02,41.489999,58.933
2,0050 (元大台灣50),2019-01-03,73.533333,13347.147
3,006208 (富邦台50),2019-01-03,40.783333,46.301
4,0050 (元大台灣50),2019-01-04,72.333333,21292.465
...,...,...,...,...
2903,006208 (富邦台50),2024-12-25,114.883334,5065.764
2904,0050 (元大台灣50),2024-12-26,198.250000,4803.561
2905,006208 (富邦台50),2024-12-26,114.966667,5522.500
2906,0050 (元大台灣50),2024-12-27,198.416667,6987.417
